In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [4]'.</span>

An Exception was encountered at 'In [4]'.

# Clustering networks

In [1]:
import pickle
import warnings
warnings.filterwarnings('ignore')
from matplotlib.axes._axes import _log
_log.setLevel('ERROR')

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random as rn
from collections import Counter
from hdbscan import HDBSCAN
from matplotlib.cm import get_cmap
from sklearn.manifold import spectral_embedding
from umap import UMAP

%matplotlib inline

seed = 42
rn.seed(seed)
np.random.seed(seed)
random_state = np.random.RandomState(seed)

Defining the network:

In [2]:
# G_ = nx.read_edgelist("data/air_traffic.edgelist")
# G_ = nx.read_edgelist("data/academic_collaboration.edgelist")
# G_ = nx.dorogovtsev_goltsev_mendes_graph(7)
with open("data/imdb.p", "rb") as f:
    G_ = nx.Graph()
    G_.add_edges_from(pickle.load(f))

print(f"largest connected components: "
      f"{[len(cc) for cc in sorted(nx.connected_components(G_), key=len, reverse=True)[:5]]}")

n_largest_cc = 1
G = G_.subgraph(set.union(*sorted(nx.connected_components(G_), key=len, reverse=True)[:n_largest_cc])).copy()
G_.clear()

largest connected components: [77744, 121, 100, 76, 51]


Reducing dimensions with Spectral Embedding and UMAP and clustering with HDBSCAN:

In [3]:
adj_matrix = nx.to_numpy_array(G)
node_list = list(G.nodes())

embedding = spectral_embedding(adj_matrix, n_components=800, drop_first=False, random_state=random_state)
embedding = UMAP(n_components=2, n_neighbors=30, min_dist=0.0, random_state=random_state).fit_transform(embedding)
clusters = HDBSCAN(min_samples=1200, min_cluster_size=800, core_dist_n_jobs=-1).fit_predict(embedding)

print(f"nodes: {len(node_list)}")
counter = Counter(clusters)
print(f"clusters: {len(counter) - 1}")
print(f"cluster sizes: {sorted(counter.items(), key=lambda x: x[1], reverse=True)}")
print(f"noise level: {np.round(counter[-1] / len(clusters), 3)}")

nodes: 77744
clusters: 22
cluster sizes: [(-1, 28718), (21, 8041), (18, 7161), (19, 4424), (13, 2669), (5, 2478), (9, 2222), (0, 2143), (8, 1965), (15, 1924), (7, 1912), (17, 1812), (11, 1740), (4, 1357), (12, 1258), (16, 1254), (1, 1195), (3, 1047), (10, 988), (20, 944), (14, 846), (2, 838), (6, 808)]
noise level: 0.369


In [4]:
cmap = get_cmap("tab20").colors
cmap = [cmap[i] for i in range(len(cmap)) if i % 2 == 0] + [cmap[i] for i in range(len(cmap)) if i % 2 == 1]
node_colors = [cmap[cluster] for cluster in clusters]

pos_fr = nx.fruchterman_reingold_layout(G, seed=seed)
pos_su = {node_list[i]: embedding[i] for i in range(len(node_list))}

def draw_network(G, pos, node_colors, pdf_name, edge_width=.1, font_size=1):
    # degrees = [x[1] for x in nx.degree(G)]
    # node_sizes = [50 * x / max(degrees) + 1 for x in degrees]
    nx.draw_networkx_nodes(G, pos=pos, node_size=1, node_color=node_colors)
    nx.draw_networkx_edges(G, pos=pos, width=edge_width)
    # nx.draw_networkx_labels(G, pos=pos, font_size=font_size)
    plt.savefig(f"results/{pdf_name}.pdf")
    plt.show()

draw_network(G, pos=pos_su, node_colors=node_colors, pdf_name="graph_su_4")
draw_network(G, pos=pos_fr, node_colors=node_colors, pdf_name="graph_fr_4")

IndexError: list index out of range

Removing nodes labeled as noise by HDBSCAN:

In [ ]:
G.remove_nodes_from([node_list[i] for i in range(len(node_list)) if clusters[i] == -1])
node_colors = [cmap[cluster] for cluster in clusters if cluster != -1]

In [ ]:
draw_network(G, pos=pos_su, node_colors=node_colors, pdf_name="graph_su_clean_4")
draw_network(G, pos=pos_fr, node_colors=node_colors, pdf_name="graph_fr_clean_4")